# Dim_page and product / provider / channel / category| issues checks

Background is that we've had issues as we add Falcon and the providers, product and channels change when in practice they're all the same

In [101]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

from data_warehouse_querying import DataWarehouseQuery
import data_parsing

In [3]:
num_days_to_query = 1
#from_datetime = datetime.now() - timedelta(days = 5)
#from_datetime = datetime(year=2020, month=1, day=4)
#to_datetime = from_datetime+ timedelta(days=num_days_to_query)
to_datetime = datetime.now().date() - timedelta(days=1) #datetime(year=2020, month=3, day=1)
from_datetime = to_datetime - timedelta(days=num_days_to_query)


In [153]:
pd.set_option("max_columns", 100)

In [4]:
dq = DataWarehouseQuery()
dq.connect()

# Get Some Basic Data

In [5]:
dq.query("select table_name from information_schema.tables where table_catalog = 'data_warehouse' and table_schema='public' order by table_name")

Starting query at 2020-04-13T07:26:13.942877
Query took 0.11


,table_name
0,bridge_anon_user_experiments
1,bridge_anon_user_experiments_backup
2,business_datamart_actions
3,business_datamart_cost
4,business_datamart_revenue
5,business_datamart_sessions
6,business_datamart_target
7,dim_activity
8,dim_activity_type
9,dim_anonymous_user


In [6]:
columns = dq.query("select t.table_name, column_name, ordinal_position, data_type, is_nullable  from information_schema.columns c left join information_schema.tables  t on t.table_name = c.table_name where t.table_schema='public'")

Starting query at 2020-04-13T07:26:14.076859
Query took 0.07


In [7]:
columns[columns.column_name.str.contains("agent")]

,table_name,column_name,ordinal_position,data_type,is_nullable


In [53]:
pages_only = dq.query("select * from dim_page")

Starting query at 2020-04-13T07:44:19.472288
Query took 5.19


In [55]:
pages_only.head(4)

,page_id,page_url,channel_id,site_version_id,sys_inserted,sys_updated,language_id,page_url_decoded,product_category_id,product_id,provider_id,page_type_id
0,1374366,www.moneysmart.sg/travaviva%20mindefel-insurance,1,1,2019-04-10 19:33:36.340630,2019-04-10 19:33:36.340630,1,None,NaN,NaN,NaN,12
1,1175910,blog.moneysmart.sg/latest-iphone-samsung-galax...,1,2,2019-02-22 19:34:25.146681,2019-02-22 19:34:25.146681,1,None,NaN,NaN,NaN,9
2,1182764,www.moneysmart.sg/Best/Card,1,1,2019-02-24 19:34:21.583005,2019-02-24 19:34:21.583005,1,None,NaN,NaN,NaN,12
3,1630997,blog.moneysmart.sg/dining/durian-delivery-sing...,1,2,2019-06-08 19:36:36.240131,2019-06-08 19:36:36.240131,1,None,NaN,NaN,NaN,9


In [58]:
page_types = dq.query("select * from dim_page_type")

Starting query at 2020-04-13T07:45:01.770718
Query took 3.13


In [59]:
page_types.head()

,page_type_id,page_type,page_sub_type,sys_inserted
0,1,listing,channel_listing,2019-01-17 08:38:27.620984
1,2,listing,provider_listing,2019-01-17 08:38:27.620984
2,3,listing,category_listing,2019-01-17 08:38:27.620984
3,4,details,product_details,2019-01-17 08:38:27.620984
4,5,interstitial_page,apply,2019-01-17 08:38:27.620984


In [61]:
pages = dq.query("select * from dim_page") # using join on creates duplicate columns 

Starting query at 2020-04-13T07:51:08.120867
Query took 0.49


In [62]:
pages_with_types = pages.merge(page_types, how="left", on="page_type_id")

In [63]:
pages_with_types.head()

,page_id,page_url,channel_id,site_version_id,sys_inserted_x,sys_updated,language_id,page_url_decoded,product_category_id,product_id,provider_id,page_type_id,page_type,page_sub_type,sys_inserted_y
0,1374366,www.moneysmart.sg/travaviva%20mindefel-insurance,1,1,2019-04-10 19:33:36.340630,2019-04-10 19:33:36.340630,1,None,NaN,NaN,NaN,12,Unknown,None,2019-01-17 08:38:27.620984
1,1175910,blog.moneysmart.sg/latest-iphone-samsung-galax...,1,2,2019-02-22 19:34:25.146681,2019-02-22 19:34:25.146681,1,None,NaN,NaN,NaN,9,blog_page,blog,2019-01-17 08:38:27.620984
2,1182764,www.moneysmart.sg/Best/Card,1,1,2019-02-24 19:34:21.583005,2019-02-24 19:34:21.583005,1,None,NaN,NaN,NaN,12,Unknown,None,2019-01-17 08:38:27.620984
3,1630997,blog.moneysmart.sg/dining/durian-delivery-sing...,1,2,2019-06-08 19:36:36.240131,2019-06-08 19:36:36.240131,1,None,NaN,NaN,NaN,9,blog_page,blog,2019-01-17 08:38:27.620984
4,1220181,blog.moneysmart.sg/budgeting/singaporeans-are-...,1,2,2019-03-06 19:34:58.775229,2019-03-06 19:34:58.775229,1,None,NaN,NaN,NaN,9,blog_page,blog,2019-01-17 08:38:27.620984


In [9]:
pages.describe()

,page_id,channel_id,site_version_id,language_id,product_category_id,product_id,provider_id,page_type_id,page_type_id
count,4.773200e+04,47732.000000,47732.000000,47732.000000,1251.000000,2391.000000,3237.000000,47732.000000,47732.000000
mean,1.518807e+06,3.309436,2.776293,1.427847,189.123102,38374.566708,13913.468953,9.282473,9.282473
std,8.973784e+05,5.829751,1.266911,0.820156,101.344944,67661.687795,22714.839973,2.099774,2.099774
min,5.163150e+05,1.000000,1.000000,1.000000,106.000000,125.000000,100.000000,1.000000,1.000000
25%,7.728062e+05,1.000000,2.000000,1.000000,116.000000,4706.000000,253.000000,9.000000,9.000000
50%,1.109732e+06,1.000000,2.000000,1.000000,120.000000,4833.000000,264.000000,9.000000,9.000000
75%,2.419145e+06,1.000000,4.000000,1.000000,296.000000,5633.000000,45080.000000,10.000000,10.000000
max,2.971890e+06,33.000000,6.000000,3.000000,1418.000000,188027.000000,62745.000000,14.000000,14.000000


In [10]:
providers = dq.query("select * from dim_provider")

Starting query at 2020-04-13T07:26:21.576865
Query took 4.82


In [11]:
providers.describe()

,provider_id,source_provider_id,status,channel_id,country_id,language_id
count,496.000000,109.000000,496.000000,496.000000,496.000000,496.000000
mean,42212.286290,61.495413,0.923387,13.834677,1.413306,1.495968
std,20759.648315,92.890779,0.266245,7.363361,0.492924,0.864557
min,99.000000,1.000000,0.000000,1.000000,1.000000,1.000000
25%,45139.000000,7.000000,1.000000,6.000000,1.000000,1.000000
50%,45788.000000,18.000000,1.000000,16.000000,1.000000,1.000000
75%,57344.000000,63.000000,1.000000,20.000000,2.000000,1.000000
max,62753.000000,292.000000,1.000000,33.000000,2.000000,3.000000


In [12]:
products = dq.query("select * from dim_product")

Starting query at 2020-04-13T07:26:26.434918
Query took 3.83


In [13]:
products.describe()

,product_id,status,language_id,channel_id,provider_id,country_id
count,819.000000,819.000000,819.000000,819.000000,819.000000,819.000000
mean,94675.642247,0.914530,1.332112,14.694750,24119.881563,1.329670
std,87504.912959,0.279751,0.744717,6.961621,25342.323393,0.470381
min,372.000000,0.000000,1.000000,4.000000,100.000000,1.000000
25%,4790.500000,1.000000,1.000000,4.000000,261.000000,1.000000
50%,102778.000000,1.000000,1.000000,20.000000,1896.000000,1.000000
75%,182614.000000,1.000000,1.000000,20.000000,45775.000000,2.000000
max,188035.000000,1.000000,3.000000,33.000000,62745.000000,2.000000


In [14]:
channels = dq.query("select * from dim_channel")

Starting query at 2020-04-13T07:26:30.301168
Query took 3.40


In [15]:
channels.head()

,channel_id,channel_key,channel_name,sys_inserted,sys_updated
0,3,car-loan,Car Loan,2018-09-11 10:11:04.481062,2018-09-11 10:11:04.481062
1,7,health-insurance,Health Insurance,2018-09-11 10:11:04.481062,2018-09-11 10:11:04.481062
2,11,international-health,International Health Insurance,2018-09-11 10:11:04.481062,2018-09-11 10:11:04.481062
3,15,personal-accident-insurance,Personal Accident Insurance,2018-09-11 10:11:04.481062,2018-09-11 10:11:04.481062
4,19,savings-account,Saving Accounts,2018-09-11 10:11:04.481062,2018-09-11 10:11:04.481062


In [16]:
dq.query("select * from dim_activity_type")

Starting query at 2020-04-13T07:26:33.723084
Query took 3.56


,activity_type_id,activity_type,activity_name,sys_inserted,sys_updated
0,179,event,Card impression,2018-09-20 13:23:21.666303,2018-09-20 13:23:21.666303
1,14333,event,ColynEvent,2019-09-24 20:11:30.304026,2019-09-24 20:11:30.304026
2,13366,event,Event,2019-08-22 19:34:34.519191,2019-08-22 19:34:34.519191
3,187,event,HomeLoanEnquiry.Step2Completed,2018-09-20 13:23:21.666303,2018-09-20 13:23:21.666303
4,8648,event,LeadGeneration.ClickConversion,2019-03-07 19:32:50.379430,2019-03-07 19:32:50.379430
5,175,event,LeadGeneration.Conversion,2018-09-20 13:23:21.666303,2018-09-20 13:23:21.666303
6,1940,event,LeadGeneration.FormSubmitted,2018-09-25 19:31:28.292266,2018-09-25 19:31:28.292266
7,183,PageView,PageView,2018-09-20 13:23:21.666303,2018-09-20 13:23:21.666303
8,10840,event,Sample Event Name,2019-05-27 19:34:22.960880,2019-05-27 19:34:22.960880
9,15226,event,SampleEvent,2019-10-24 20:10:28.749646,2019-10-24 20:10:28.749646


In [17]:
dq.query("select column_name, data_type, is_nullable from information_schema.columns where table_name='fact_activities' order by ordinal_position")

Starting query at 2020-04-13T07:26:37.315351
Query took 0.06


,column_name,data_type,is_nullable
0,fact_activity_id,bigint,NO
1,activity_date_id,integer,YES
2,activity_time_id,integer,YES
3,session_id,bigint,YES
4,anonymous_user_id,bigint,YES
5,user_id,bigint,YES
6,device_id,integer,YES
7,browser_id,integer,YES
8,session_campaign_id,integer,YES
9,activity_type_id,integer,YES


In [18]:
# sample of fact_activities
query = """
select 
    fa.fact_activity_id, fa.session_id, fa.anonymous_user_id, fa.activity_type_id, fa.user_filter_type
    , fa.channel_id, fa.product_id, fa.provider_id, fa.product_category_id
    , fa.page_id, fa.page_type_id
    , dim_date.full_date 
    , dim_activity_type.activity_type, dim_activity_type.activity_name
    

from fact_activities fa
    left join dim_activity_type on fa.activity_type_id = dim_activity_type.activity_type_id
    left join dim_date on fa.activity_date_id = dim_date.date_id
    where dim_date.full_date between '{}' and '{}'
""".format(from_datetime, to_datetime)
activities = dq.query(query)

Starting query at 2020-04-13T07:26:37.390533
Query took 11.53


In [19]:
activities.describe()

,fact_activity_id,session_id,anonymous_user_id,activity_type_id,channel_id,product_id,provider_id,product_category_id,page_id,page_type_id
count,5.958460e+05,5.958460e+05,5.958460e+05,595846.000000,595846.000000,12474.000000,16932.000000,42986.000000,5.958430e+05,595843.000000
mean,2.136558e+08,7.219586e+07,5.701157e+07,8244.053024,2.733436,84810.664502,19124.628750,129.221142,1.543298e+06,9.109771
std,2.686357e+05,6.364285e+04,1.100790e+07,5914.837455,3.849261,89249.705488,24814.344249,45.470406,9.056764e+05,1.881297
min,2.130960e+08,7.205823e+07,1.017738e+07,173.000000,1.000000,372.000000,102.000000,106.000000,5.163150e+05,1.000000
25%,2.134511e+08,7.214810e+07,6.148658e+07,183.000000,1.000000,456.000000,106.000000,116.000000,7.112740e+05,9.000000
50%,2.136498e+08,7.219613e+07,6.164048e+07,10353.000000,1.000000,1980.000000,112.000000,118.000000,1.282686e+06,9.000000
75%,2.138421e+08,7.224187e+07,6.167717e+07,13685.000000,4.000000,181730.000000,45670.000000,120.000000,2.399859e+06,9.000000
max,2.142708e+08,7.233876e+07,6.175101e+07,17984.000000,34.000000,188027.000000,62745.000000,327.000000,2.971890e+06,14.000000


In [20]:
activities.head(5)

,fact_activity_id,session_id,anonymous_user_id,activity_type_id,user_filter_type,channel_id,product_id,provider_id,product_category_id,page_id,page_type_id,full_date,activity_type,activity_name
0,213841546,72197115,24272992,183,external_visitor,2,NaN,NaN,NaN,520976.0,1.0,2020-04-11,PageView,PageView
1,213806134,72124176,61575674,183,external_visitor,2,NaN,NaN,NaN,520976.0,1.0,2020-04-11,PageView,PageView
2,213343772,72129952,61621457,183,external_visitor,2,NaN,NaN,NaN,520976.0,1.0,2020-04-11,PageView,PageView
3,213479155,72130406,61596174,183,external_visitor,2,NaN,NaN,NaN,520976.0,1.0,2020-04-11,PageView,PageView
4,213841550,72197115,24272992,8613,external_visitor,2,NaN,NaN,NaN,520976.0,1.0,2020-04-11,event,UserEngagement.ButtonClick


# Check if we have duplicates in fact_activities with equivalent dim_page records

# Check no duplicate exact urls in dim_page

In [136]:
page_counts = pages.groupby(["page_url"]).count()

In [137]:
page_counts[page_counts.page_id>1]

,page_id,channel_id,site_version_id,sys_inserted,sys_updated,language_id,page_url_decoded,product_category_id,product_id,provider_id,page_type_id,canonical_url
page_url,,,,,,,,,,,,


# Check There Aren't Any AB test URLs in dim_page

There shouldn't be any with things like www3 or blog3 or www-new as they don't represent different pages.

By looking at what is in dim_page, we can see if there's likely an issue, but whether they are recent requires joining with fact_activities

In [138]:
#bad_pages = pages[pages.page_url.contains("www-new")| pages.page_url.str.contains("blog3")]
url_roots = pages_with_types["page_url"].str.split(".").str[0]
pages_with_types["root"] = url_roots

root_counts = pages_with_types.groupby("root").size()



In [139]:
expected_roots = ["blog","www", "get", "learn", "iss", "forum", "blog-admin"]

In [140]:
bad_roots = root_counts[~root_counts.keys().isin(expected_roots)]
print("Just showing top bad looking root by number of pages (not by number of pageviews / events)")
bad_roots.sort_values(ascending=False).head(20)

Just showing top bad looking root by number of pages (not by number of pageviews / events)


root
blog3                           3953
www-new                         1638
ti                               362
www3                             213
blog-staging                     172
blog-cdn                         139
ms-forms                          45
www5                              16
blog-staging-new                  16
admin                             15
cc                                14
ti-admin                          14
www-ecs                           13
pl                                11
tnp-blog-moneysmart-sg-local      11
blog-moneysmart-sg                10
ci                                 8
blog-moneysmart-hk-admin           7
blog-moneysmart-sg-admin           7
mortgage                           6
dtype: int64

In [141]:
activities_pages = pd.merge(activities, pages_with_types, on="page_id", how="left", suffixes=("_activities", "_pages"))

In [142]:
bad_page_activities = activities_pages[~activities_pages.root.isin(expected_roots)].groupby(["page_url", "activity_type", "root"]).agg("size")

In [143]:
bad_page_activities = pd.DataFrame(bad_page_activities).reset_index()

In [154]:
bad_page_activities.sort_values(0, ascending=False)

,page_url,activity_type,root,0
70,blog3.moneysmart.sg/fixed-deposits/best-fixed-...,event,blog3,30
69,blog3.moneysmart.sg/fixed-deposits/best-fixed-...,PageView,blog3,23
120,blog3.moneysmart.sg/savings-accounts/best-savi...,PageView,blog3,13
125,blog3.moneysmart.sg/savings-accounts/dbs-multi...,event,blog3,13
12,blog3.moneysmart.sg/budgeting/covid-19-financi...,event,blog3,11
81,blog3.moneysmart.sg/invest/dbs-ocbc-poems-fsmo...,event,blog3,10
19,blog3.moneysmart.sg/budgeting/cpf-retirement-sum,event,blog3,9
90,blog3.moneysmart.sg/invest/investment-brokerag...,event,blog3,9
121,blog3.moneysmart.sg/savings-accounts/best-savi...,event,blog3,9
25,blog3.moneysmart.sg/budgeting/singtel-combo-mo...,event,blog3,8


In [145]:
bad_page_activities_by_root = activities_pages[~activities_pages.root.isin(expected_roots)].groupby(["root"]).agg("size")
bad_page_activities_by_root

root
blog-cdn      1
blog3       425
www-new       2
dtype: int64

In [146]:
bad_page_activities[bad_page_activities.root.isin(["admin"])]

,page_url,activity_type,root,0


# Check No duplicate urls in dim_page based on external function to determine canonical

In brief discussion, this might not be a big issue and some applications are case sensitive.

In [147]:
pages["canonical_url"] = pages.apply(lambda x: data_parsing.get_canonical_url("https://"+x.page_url), axis=1)

In [148]:
canonical_counts = pd.DataFrame(pages.groupby(["canonical_url"]).size()).reset_index()

In [149]:
canonical_duplicates = canonical_counts[canonical_counts[0]>1].merge(pages[["canonical_url","page_id", "page_url"]], how="right", on="canonical_url")  # I'm sure there must be a neater way for all this.

In [150]:
canonical_duplicates#.sort_values(["canonical_url"])

,canonical_url,0,page_id,page_url
0,blog-cdn.moneysmart.sg/phpmyadmin,2.0,1776329,blog-cdn.moneysmart.sg/phpMyAdmin
1,blog-cdn.moneysmart.sg/phpmyadmin,2.0,1776073,blog-cdn.moneysmart.sg/phpmyadmin
2,blog-staging.moneysmart.sg/search/credit,2.0,891601,blog-staging.moneysmart.sg/search/Credit
3,blog-staging.moneysmart.sg/search/credit,2.0,891613,blog-staging.moneysmart.sg/search/credit
4,blog-staging.moneysmart.sg/search/credit+cards,2.0,891522,blog-staging.moneysmart.sg/search/credit+cards
5,blog-staging.moneysmart.sg/search/credit+cards,2.0,893211,blog-staging.moneysmart.sg/search/Credit+cards
6,blog.moneysmart.hk,2.0,1015130,blog.moneysmart.hk
7,blog.moneysmart.hk,2.0,2965738,blog3.moneysmart.hk
8,blog.moneysmart.hk/air-miles,2.0,2691430,blog.moneysmart.hk/air-miles
9,blog.moneysmart.hk/air-miles,2.0,2966806,blog3.moneysmart.hk/air-miles


# If there are AB URLs, check that they have the same product etc information

In [167]:
can_counts = pd.DataFrame(pd.DataFrame(pages.groupby(["canonical_url", "product_id", "provider_id", "product_category_id"]).size()).groupby(["canonical_url"]).size()).reset_index()

In [171]:
can_counts[can_counts[0]>1]

,canonical_url,0


In [169]:
ccm = can_counts.merge(pages[["canonical_url", "product_id", "provider_id", "product_category_id", "page_url"]], how="right", on="canonical_url")

In [170]:
ccm[ccm[0]>1]

,canonical_url,0,product_id,provider_id,product_category_id,page_url


# Check page_type_id in fact_activities matches with dim_page

In [91]:
activities_pages.columns

Index(['fact_activity_id', 'session_id', 'anonymous_user_id',
       'activity_type_id', 'user_filter_type', 'channel_id_activities',
       'product_id_activities', 'provider_id_activities',
       'product_category_id_activities', 'page_id', 'page_type_id_activities',
       'full_date', 'activity_type', 'activity_name', 'page_url',
       'channel_id_pages', 'site_version_id', 'sys_inserted_x', 'sys_updated',
       'language_id', 'page_url_decoded', 'product_category_id_pages',
       'product_id_pages', 'provider_id_pages', 'page_type_id_pages',
       'page_type', 'page_sub_type', 'sys_inserted_y', 'root'],
      dtype='object')

In [93]:
type_cols_in_activities_pages = [z for z in activities_pages.columns if "page_type" in z]
print("grouping  by "+", ".join(type_cols_in_activities_pages))
ap_g1 = activities_pages.groupby(["page_id",]+ type_cols_in_activities_pages).count().reset_index()
ap_g2 = ap_g1.groupby(["page_id"]).count().reset_index()
ap_g2[ap_g2.page_type_id_pages>1]

grouping  by page_type_id_activities, page_type_id_pages, page_type


,page_id,page_type_id_activities,page_type_id_pages,page_type,fact_activity_id,session_id,anonymous_user_id,activity_type_id,user_filter_type,channel_id_activities,...,sys_inserted_x,sys_updated,language_id,page_url_decoded,product_category_id_pages,product_id_pages,provider_id_pages,page_sub_type,sys_inserted_y,root


# Check that the slug in the dim_page is unique by country & language

In [196]:
group_cols = ["provider_name", "channel_id", "country_id", "language_id"]
pg = providers.groupby([ "slug", ]+ group_cols).size().reset_index()
#pg.reset_index().groupby(["slug"]).size()
pg[pg[0]>1]
#.groupby(["slug"]).size().reset_index().rename(columns={0:"num"})

,slug,provider_name,channel_id,country_id,language_id,0


# Provider Slugs Don't Include Spaces

In [197]:
providers[providers.slug.str.contains(" ")]

,provider_id,provider_name,sys_inserted,sys_updated,source_provider_id,slug,status,channel_id,country_id,language_id
35,62153,Standard Chartered,2020-03-18 08:48:51.008899,2020-03-18 08:48:51.008899,NaN,Standard Chartered,1,19,1,1
36,62169,China Construction Bank (Asia),2020-03-18 08:48:51.008899,2020-03-18 08:48:51.008899,NaN,China Construction Bank (Asia),1,16,2,3
47,56815,Hang Seng Bank,2020-03-18 08:48:51.008899,2020-03-18 08:48:51.008899,NaN,Hang Seng Bank,1,25,2,3
55,45088,Hang Seng Bank,2020-03-18 08:48:51.008899,2020-03-18 08:48:51.008899,NaN,Hang Seng Bank,1,16,2,3
58,45136,China Construction Bank (Asia),2020-03-18 08:48:51.008899,2020-03-18 08:48:51.008899,NaN,China Construction Bank (Asia),1,25,2,3
59,45152,Bank Of China,2020-03-18 08:48:51.008899,2020-03-18 08:48:51.008899,NaN,Bank of China,1,25,2,3
66,62297,Tokio Marine,2020-03-18 08:58:37.783151,2020-03-18 08:58:37.783151,NaN,Tokio Marine,1,15,1,1
67,62313,Century Tokyo Leasing,2020-03-18 08:58:37.783151,2020-03-18 08:58:37.783151,NaN,Century Tokyo Leasing,1,3,1,1
69,45256,Standard Chartered,2020-03-18 20:09:00.555460,2020-03-18 20:09:00.555460,NaN,Standard Chartered,1,6,1,1
70,62321,Hong Leong Finance,2020-03-18 20:09:00.555460,2020-03-18 20:09:00.555460,NaN,Hong Leong Finance,1,10,1,1


# Slugs are case insensitive

In [204]:
# sense check of table below
providers[providers.provider_id.isin([45766, 46190, 62745, 57395, 46242, 267, 57335 ])]

,provider_id,provider_name,sys_inserted,sys_updated,source_provider_id,slug,status,channel_id,country_id,language_id,slug_lowered
34,45766,Aig,2020-03-18 08:48:51.008899,2020-03-18 08:48:51.008899,NaN,AIG,1,2,1,1,aig
95,46190,Aig,2020-03-24 02:15:56.201027,2020-03-24 02:15:56.201027,NaN,aig,1,9,2,3,aig
118,62745,Aig,2020-03-28 09:04:47.257312,2020-03-28 09:04:47.257312,NaN,aig,1,9,1,1,aig
230,57395,Aig,2020-03-24 02:15:56.201027,2020-03-24 02:15:56.201027,NaN,aig,1,2,1,1,aig
347,46242,Aig,2020-03-24 02:15:56.201027,2020-03-24 02:15:56.201027,NaN,aig,1,9,2,1,aig
376,267,AIG,2019-02-01 01:43:57.318499,2019-02-01 01:43:57.318499,244.0,aig,1,20,1,1,aig
455,57335,Aig,2020-03-20 07:39:44.500416,2020-03-20 07:39:44.500416,NaN,aig,1,20,2,1,aig


In [209]:
providers["slug_lowered"] = providers.slug.str.lower()
group_cols = ["provider_name", "channel_id", "country_id", "language_id"]
pg = providers.groupby([ "slug_lowered", ]+ group_cols).size().reset_index().rename(columns={0: "num"})
#pg.reset_index().groupby(["slug"]).size()
pgm = pg.merge(providers[["slug_lowered", "slug", "provider_id", "sys_inserted"]+ group_cols], how="right", on=(["slug_lowered",]+group_cols))#[pg[0]>1]
pgm[pgm.num>1]

,slug_lowered,provider_name,channel_id,country_id,language_id,num,slug,provider_id,sys_inserted
12,aig,Aig,2,1,1,2,AIG,45766,2020-03-18 08:48:51.008899
13,aig,Aig,2,1,1,2,aig,57395,2020-03-24 02:15:56.201027
36,aviva,Aviva,2,1,1,2,Aviva,62285,2020-03-18 08:48:51.008899
37,aviva,Aviva,2,1,1,2,aviva,46230,2020-03-24 02:15:56.201027
40,axa,Axa,2,1,1,2,AXA,45762,2020-03-18 08:48:51.008899
41,axa,Axa,2,1,1,2,axa,45660,2020-03-24 02:15:56.201027
44,axa,Axa,15,1,1,2,axa,57367,2020-03-24 02:15:56.201027
45,axa,Axa,15,1,1,2,AXA,56867,2020-03-18 08:48:51.008899
78,bea,Bea,16,2,1,2,bea,45608,2020-03-20 07:24:48.737590
79,bea,Bea,16,2,1,2,BEA,57307,2020-03-20 07:24:48.737590


In [214]:
#df = pgm[pgm.num>1].merge(pages, how="left", on="provider_id").merge(activities, how="right", on="page_id") #.groupby([["slug_lowered", "slug", "provider_id"]]).size()

In [223]:
#df.head()
#df = activities.merge(pgm, how="left", on="provider_id")#.merge(pgm, how="left", on="provider_id")
#df2 = df.groupby(["slug_lowered", "slug", ] + group_cols).size().reset_index().rename({0:"num"})

In [224]:
#df2[df2["num"]>1]